# Data Transformations

Looks into how to transform data to go into a neural network later

In [1]:
import librosa 
import os
#The path to the audio files
path = os.getcwd()+'\\Joji\\'

#Loads in the first track as a test
#x is the audio, sr is the sampling rate
x,sr = librosa.load(path+'01 ATTENTION.mp3')

print(type(x),type(sr))

C:\Users\Jasn\.conda\envs\audio-gan\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


<class 'numpy.ndarray'> <class 'int'>


In [142]:
#x1 = x[0:220500]
x1

array([-2.3970488e-05, -1.0427811e-05, -3.9347137e-06, ...,
       -3.3414442e-02, -2.2057846e-02, -1.2235547e-02], dtype=float32)

First I'm just going to give the signal a try completely unprocessed. Just run it through a simple, untrained neural network. The descriminator in the GAN would need to do that anyway (though I'd probably train it first).

In [143]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

model = keras.Sequential()

model.add(layers.Dense(1000,activation='relu'))#,input_shape=(1,len(x1))))
model.add(layers.Dense(500,activation='relu'))
model.add(layers.Dense(1,activation='relu'))

#model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.5),metrics=["accuracy"])



In [144]:
#Just running the audio through an untrained neural network
y = model.predict(x1)
#Since every value is output as an array with a single entry, pull it out of the array
y = [val[0] for val in y]

In [145]:
import IPython.display as ipd

#ipd.Audio(path+'01 ATTENTION.wav')
ipd.Audio(y,rate=sr)
#model.get_weights()

Depending on the song used, this probably sounds a lot like it but severely distorted. That's an issue with the way that the data is being loaded into Keras. It is currently taking the input shape to be 1 rather than the number of samples that make up the song. I'm getting a weird error when I try to get it to take in the whole song at once.

## Wavelet transforms

Since Fourier transforms assume a stationary signal, and songs are very non-stationary signals, I am going to try a wavelet transform instead. The most important feature of any feature extraction here is invertibility. If I can't invert the transformation, then any output from the generator will be worthless.

In [23]:
import pywt

y = pywt.dwt(x,'db1',mode='symmetric')

In [37]:
pywt.scale2frequency('gaus1',5)/(1/sr)

882.0

In [14]:
w = pywt.Wavelet('db1')
pywt.dwt_max_level(data_len=len(x), filter_len=w.dec_len)
#pywt.dwt_max_level(1000, w)
#pywt.dwt_max_level(1000, 'sym5')

21

In [31]:
pywt.wavelist(kind='continuous')

['cgau1',
 'cgau2',
 'cgau3',
 'cgau4',
 'cgau5',
 'cgau6',
 'cgau7',
 'cgau8',
 'cmor',
 'fbsp',
 'gaus1',
 'gaus2',
 'gaus3',
 'gaus4',
 'gaus5',
 'gaus6',
 'gaus7',
 'gaus8',
 'mexh',
 'morl',
 'shan']